In [1]:

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
import pandas as pd
import warnings
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from genepro.scikit import GeneProRegressor
from genepro.bagging import Bagging

from genepro.node_impl import *
from joblib import cpu_count
from genepro.util import compute_linear_scaling
from ensembles.GPforest import GPForestRegressor
from ensembles.GPBoosting import GPGBRegressor
import sympy
from scipy.stats import pearsonr
df = pd.read_csv('data/markers_filtered_adj_imp.csv', index_col=0)
targets = pd.read_csv('data/pheno_full_adj.csv', index_col=0)
df.index = [i for i in range(len(df))]
targets.index = [i for i in range(len(targets))]
warnings.filterwarnings("ignore")
# targets = targets.reset_index()
# df = df.reset_index()
X, y = df, targets['Длина.третьего.подфлагового.листа..мм']
X = X.dropna()
y = y.dropna()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
use_linear_scaling=True

In [ ]:
gp_forest = Bagging(use_linear_scaling=use_linear_scaling, num_of_trees=1,# linear scaling applies a linear layer to the prediction (intercept + slope*prediction) 
  evo_kwargs={'verbose': False, 'pop_size': 128, 'max_gens': 10, 'max_tree_size': 300, 'n_jobs': -1, 'parallel_prefer': 'threads'})
gp_forest.fit(X, y)
test_mse1 = mean_squared_error(y_test, gp_forest.predict(X_test))
print("FOREST The mean squared error on the test set is {:.3f} (respective R^2 score is {:.3f})".format(
  test_mse1, 1 - test_mse1/np.var(y_train)))

In [ ]:
pearson_forest_test = pearsonr(y_test, gp_forest.predict(X_test))
pearson_forest_train = pearsonr(y_train, gp_forest.predict(X_train))
print(f'train: {pearson_forest_train} test {pearson_forest_test}')